## I want a simple, clear way to see how well my model is doing. Let's compare the Ground Truth EF with ones we can calculate from our Model.

We'll try to go through all videos, compute their estimated EF's and see how they compare with the ground truths.

I think we'll try to do a Mean Squared Error, and we can also simply just look at the average percentage error:<br>
$Y_i$ is the predicted value.<br>
$\hat{Y_i}$ is the expected value.

$$ MSE = \frac{1}{n}\sum_{i = 1}^n (Y_i - \hat{Y_i})^2 $$
$$ \% error = \frac{|\hat{Y_i} - Y_i|}{\hat{Y_i}} $$

In [1]:
model_name_1 = ""
model_name_2 = ""

SyntaxError: EOL while scanning string literal (<ipython-input-1-42b272d6f93c>, line 1)

In [2]:
import os 
os.chdir("..")
print(os.getcwd())

/home/wang/workspace/JupyterNoteBooksAll/fully-automated-multi-heartbeat-echocardiography-video-segmentation-and-motion-tracking


In [2]:
%config Completer.use_jedi = False

import echonet
from echonet.datasets import Echo

import torch.nn.functional as F
from torchvision.models.video import r2plus1d_18
from torch.utils.data import Dataset, DataLoader, Subset
from multiprocessing import cpu_count

from src.utils.torch_utils import TransformDataset, torch_collate
from src.utils.echo_utils import get2dPucks
from src.utils.camus_validate import cleanupSegmentation
from src.transform_utils import generate_2dmotion_field
from src.visualization_utils import categorical_dice
from src.loss_functions import huber_loss, convert_to_1hot, convert_to_1hot_tensor
from src.model.R2plus1D_18_MotionNet import R2plus1D_18_MotionNet
from src.echonet_dataset import EchoNetDynamicDataset

import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import random
import pickle
import time

tic, toc = (time.time, time.time)

## Load in Test data to validate

In [3]:
batch_size = 4
num_workers = max(4, cpu_count()//2)

def worker_init_fn_valid(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)
    

def worker_init_fn(worker_id):
    # See here: https://pytorch.org/docs/stable/notes/randomness.html#dataloader
    # and the original post of the problem: https://github.com/pytorch/pytorch/issues/5059#issuecomment-817373837
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    

def permuter(list1, list2):
    for i1 in list1:
        for i2 in list2:
            yield (i1, i2)
            

param_trainLoader = {'collate_fn': torch_collate,
                     'batch_size': batch_size,
                     'num_workers': max(4, cpu_count()//2),
                     'worker_init_fn': worker_init_fn}

param_testLoader = {'collate_fn': torch_collate,
                    'batch_size': batch_size,
                    'shuffle': False,
                    'num_workers': max(4, cpu_count()//2),
                    'worker_init_fn': worker_init_fn}

paramLoader = {'train': param_trainLoader,
               'valid': param_testLoader,
               'test':  param_testLoader}

In [4]:
with open("fold_indexes/stanford_valid_sampled_indices", "rb") as infile:
    valid_mask = pickle.load(infile)
infile.close()

full_dataset = EchoNetDynamicDataset(split='val', clip_length="full", raise_for_es_ed=False, subset_indices=valid_mask, period=1)
test_dataset = EchoNetDynamicDataset(split='test', clip_length="full", raise_for_es_ed=False, period=1)

100%|██████████| 16/16 [00:01<00:00, 13.84it/s]


In [5]:
print("vids in test dataset:", len(test_dataset)) # how many videos are in the test dataset

test_pat_indeces = [i for i in range(len(test_dataset))] # creates a list of all indeces of test_dataset
# print(len(test_pat_index))

# print(test_pat_indeces)
# video, (filename, EF, es_clip_index, ed_clip_index, es_index, ed_index, es_frame, ed_frame, es_label, ed_label) = test_dataset[test_pat_index]

vids in test dataset: 1276


In [6]:
def get_values_for_curr_vid(ind, test_dataset):
    video, (filename, GROUND_TRUTH_EF, es_clip_index, ed_clip_index, es_index, ed_index, es_frame, ed_frame, es_label, ed_label) = test_dataset[ind]
    return video, (filename, GROUND_TRUTH_EF, es_clip_index, ed_clip_index, es_index, ed_index, es_frame, ed_frame, es_label, ed_label)

## Load in Trained Model to Test

In [7]:
model_save_path = "save_models/R2plus1DMotionSegNet_model.pth"

model = torch.nn.DataParallel(R2plus1D_18_MotionNet())
model.to("cuda")
torch.cuda.empty_cache()
model.load_state_dict(torch.load(model_save_path)["model"])
print(f'R2+1D MotionNet has {sum(p.numel() for p in model.parameters() if p.requires_grad)} parameters.')

model.eval();

R2+1D MotionNet has 31575731 parameters.


## Define Funcs to Calculate our Values for Assessment

In [8]:
def compute_predicted_ef(ed_label, es_label):
    # Use the ground true lables to derive the EF
    output_ED = ed_label
    output_ES = es_label

    # Use the Simpson's Monoplane method
    length_ed, radius_ed = get2dPucks((output_ED == 1).astype('int'), (1.0, 1.0))
    length_es, radius_es = get2dPucks((output_ES == 1).astype('int'), (1.0, 1.0))

    edv = np.sum(((np.pi * radius_ed * radius_ed) * length_ed / len(radius_ed)))
    esv = np.sum(((np.pi * radius_es * radius_es) * length_es / len(radius_es)))

    ef_predicted = (edv - esv) / edv * 100
    return ef_predicted

In [9]:
def run(video_indeces, test_dataset):
    Error_Percentages = []
    Predicted_Values = []
    Expected_Values = [] 
    # get predicted and ground truth EF values
    for ind in video_indeces:
        video, (filename, GROUND_TRUTH_EF, es_clip_index, ed_clip_index, es_index, ed_index, es_frame, ed_frame, es_label, ed_label) = get_values_for_curr_vid(ind, test_dataset)
        ef_predicted = compute_predicted_ef(ed_label, es_label)
        Predicted_Values.append(ef_predicted)
        Expected_Values.append(GROUND_TRUTH_EF)
    # compute MSE and error percentages
    tmp = []
    for pred, exp in zip(Predicted_Values, Expected_Values):
        # for MSE
        tmp.append((pred - exp) ** 2)
        # for error percentages
        Error_Percentages.append(abs(exp - pred)/exp)
    # finish computing MSE
    MSE = (1/len(tmp)) * np.sum(tmp)
    
    # return values
    return MSE, Error_Percentages, Predicted_Values, Expected_Values

## Compute, Display, and Save to File the Results

In [15]:
MSE, Error_Percentages, Predicted_Values, Expected_Values = run(test_pat_indeces, test_dataset)

In [16]:
print(f'MSE: {MSE}')
avg_error_perc = np.sum([(1/len(Error_Percentages) * err) for err in Error_Percentages])
print(f"Average Error_Percentages: {avg_error_perc}")

MSE: 7.169083941314519
Average Error_Percentages: 0.031004062276521215


In [17]:
print("Observed\tExpected\t% Difference\n")
count = 0
for pred, exp, per_diff in zip(Predicted_Values, Expected_Values, Error_Percentages):
    count += 1
    if count == 10:
        break
    print(f"{pred:.5f}\t{exp:.5f}\t{per_diff:.5f}")

Observed	Expected	% Difference

57.10036	55.95179	0.02053
40.61877	41.01442	0.00965
50.43640	50.79472	0.00705
52.61210	50.63498	0.03905
63.55931	63.09809	0.00731
59.80891	58.49444	0.02247
67.73466	67.38157	0.00524
57.50785	57.28491	0.00389
58.71777	59.91861	0.02004


In [18]:
# save results to a file for now.
save_filename = "./warren-random/save_EF-2.txt"
with open(save_filename, "w") as file:
    file.write(f'Model Name/Path: {model_save_path}\n')
    file.write("####### Basic Stats: #######\n\n")
    file.write(f'Num of Vids: {len(Expected_Values)}\n')
    file.write(f"MSE: {MSE}\n")
    file.write(f"Average Percentage Error: {avg_error_perc}\n")
    file.write('####### Observed, Expected, and % Difference #######\n\n') 
    for pred, exp, per_diff in zip(Predicted_Values, Expected_Values, Error_Percentages):
        file.write(f"{pred}\t{exp}\t{per_diff}\n")

# FIXME: This is just using numbers from the test dataset itself. The model is not doing anything to compute the Ejection Fraction from how I've written it.

## We need to pass in clips to the model, and it will output as according to this single line of code:
`segmentation_output, motion_output = model(torch.Tensor(one_clip))`